<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tim Harrison
Exam 1
Student ID: 002-24-5948

In [16]:
#Handling file imports here and creating dataframes
from os import walk
import numpy as np
import random
random.seed(12345)

_, _, filenames_neg = next(walk("TRAINING/negative"))
_, _, filenames_pos = next(walk("TRAINING/positive"))
#print(filenames_neg[0:10])
ones = [1] * len(filenames_pos)
zeroes = [0] * len(filenames_neg)
pos_txt = []
for name in filenames_pos:
  with open('/content/TRAINING/positive/'+name, 'r') as file:
      d = file.read().replace('\n', ' ')
      pos_txt.append(d)
df_pos = pd.DataFrame(data={'text': pos_txt, 'sentiment': ones})
neg_txt = []
for name in filenames_neg:
  with open('/content/TRAINING/negative/'+name, 'r') as file:
      d = file.read().replace('\n', ' ')
      neg_txt.append(d)
df_neg = pd.DataFrame(data={'text': neg_txt, 'sentiment': zeroes})

#df_neg.sample(frac=0.75,random_state=12345)

data = df_pos.append(df_neg)
#data.sample(frac=0.75,random_state=12345)

,text,sentiment
189,"Okay, now what the hell is this supposed to be...",0
1498,The only reason I wanted to see this was becau...,1
826,"There are few comedies like this, where almost...",1
850,This is a very good movie. Do you want to know...,1
748,"""Margaritas and Cock...""<br /><br />This treme...",1
...,...,...
1527,My choice for greatest movie ever used to be L...,1
1370,"The Haunted World of Edward D. Wood, Jr. isn't...",1
515,Perhaps the funniest 'backstage at Hollywood' ...,1
883,This is a powerful documentary about domestic ...,1


Question 1)

In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, FeatureUnion

def create_model(alg_name, vectorization, train):
  if alg_name == "naive_bayes":
    learn = MultinomialNB()
  elif alg_name == "perceptron":
    learn = Perceptron()
  elif alg_name == "logistic_regression":
    learn = LogisticRegression()
  elif alg_name == "random_forest":
    learn = RandomForestClassifier()
  elif alg_name == "svm":
    learn = SVC()
  else:
    raise NameError('incorrect algorithm name')

  if vectorization == "tfidf":
    vect = TfidfVectorizer()
  elif vectorization == "bag_of_words":
    vect = CountVectorizer()
  elif vectorization == "both":
    vect = FeatureUnion([("tfidf", TfidfVectorizer()), ("bag_of_words", CountVectorizer())])
  else:
    raise NameError('incorrect vectorization name')
  
  model = make_pipeline(vect, learn)
  model.fit(train.text, train.sentiment)
  return model


Question 2)

In [42]:
from sklearn.model_selection import train_test_split
train_a, test_a = train_test_split(data, test_size=0.25,random_state=12345)
model_a = create_model("naive_bayes","both",train_a)

train_b, test_b = train_test_split(data, test_size=0.30,random_state=12345)
model_b = create_model("random_forest","both",train_b)

train_c, test_c = train_test_split(data, test_size=0.40,random_state=12345)
model_c = create_model("svm","both",train_c)


Question 3)

In [45]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
labels_a = model_a.predict(test_a.text)
results_a = pd.DataFrame(data={'text': test_a.text, 'true_sentiment': test_a.sentiment, 'modeled_sentiment': labels_a})

scores_a = pd.DataFrame(data={'model': 'model_a', 'training_portion': [0.75], 'test_portion': [0.25], 'accuracy':accuracy_score(test_a.sentiment,labels_a),'precision':precision_score(test_a.sentiment,labels_a),'recall':recall_score(test_a.sentiment,labels_a),'f1-score':f1_score(test_a.sentiment,labels_a)})


labels_b = model_b.predict(test_b.text)
results_b = pd.DataFrame(data={'text': test_b.text, 'true_sentiment': test_b.sentiment, 'modeled_sentiment': labels_b})

scores_b = pd.DataFrame(data={'model': 'model_b', 'training_portion': [0.70], 'test_portion': [0.30], 'accuracy':accuracy_score(test_b.sentiment,labels_b),'precision':precision_score(test_b.sentiment,labels_b),'recall':recall_score(test_b.sentiment,labels_b),'f1-score':f1_score(test_b.sentiment,labels_b)})


labels_c = model_c.predict(test_c.text)
results_c = pd.DataFrame(data={'text': test_c.text, 'true_sentiment': test_c.sentiment, 'modeled_sentiment': labels_c})

scores_c = pd.DataFrame(data={'model': 'model_c', 'training_portion': [0.60], 'test_portion': [0.40], 'accuracy':accuracy_score(test_c.sentiment,labels_c),'precision':precision_score(test_c.sentiment,labels_c),'recall':recall_score(test_c.sentiment,labels_c),'f1-score':f1_score(test_c.sentiment,labels_c)})

scores = scores_a.append(scores_b)
scores = scores.append(scores_c)
scores

,model,training_portion,test_portion,accuracy,precision,recall,f1-score
0,model_a,0.75,0.25,0.861264,0.856828,0.915294,0.885097
0,model_b,0.70,0.30,0.804124,0.780366,0.923228,0.845807
0,model_c,0.60,0.40,0.743127,0.732034,0.883824,0.800799
